## Lecture 4 Aggregating data

### Aggregate opertions

**Concept**: 
*Aggregation* is the act of taking multiple values and reducing them down to a single value. Two of the simplest forms of aggregation are counting and adding.

- Counting 
- Adding
- Computing the average (mean)
- Finding the minimum (lowest value)
- Finding the maximum (highest value)

Aggregate expressions: combine values from multiple rows <br>
Non-Aggregate or scalar expressions: return one value per row

Be **careful** when combining aggregate expressions and scalar expressions!!!



#### Common aggregate functions

Function Name|Function Description|Example Invocation|
-|-|-
`COUNT`|Count all rows|`COUNT(*)`
`SUM`|Add all supplied values and return result|`SUM(salary)`
`AVG`|Return the average of all supplied values|`AVG(salary)`
`MIN`|Return the lowest value|`MIN(salary)`
`MAX`|Return the highest value|`MAX(salary)`

Some data analysts use the expression `SUM(1)` instead of `COUNT(*)`. 

These two aggregate expressions do the same thing: they count the number of rows in a table. This is because when you use a scalar argument to an aggregate function, the the aggregate function aggregates that same value over all the rows. 


#### Using aggregrate functions in the `SELECT` statement

Uisng aggregrate functions in select list
```MySQL
SELECT COUNT(*) AS num_rows FROM employees;
SELECT SUM(salary) AS total_salary FROM employees;
SELECT MIN(salary), MAX(salary) FROM employees;
SELECT round(SUM(salary) * 0.062) AS total_tax FROM employees;
```

- Valid mixing of aggregate and scalar operations ✅
```SQL
round(AVG(list_price))
SUM(salary * 0.062)         
```    
- INVALID queries ❌
```SQL 
SELECT salary - AVG(salary) FROM employees;
SELECT first_name, SUM(salary) FROM employees;
```

**Conditional aggregation**

```SQL
SELECT COUNT(*) FROM employees WHERE salary > 30000;
```

Notice that you **CANNOT** use aggregate functions in `WHERE` clause.

因为 `WHERE` caluse 后面接的布尔表达式将对每一行进行判断。

### The `GROUP BY` clause

上面的 aggregation 返回的都是一行，利用 `GROUP BY` 可以对数据分组并对每组进行 aggregation, 然后返回多行。

Some examples
```SQL
-- How many employees are there in total?
SELECT COUNT(*) FROM employees;

-- How many employees are there in each office?
-- Return one row per group
SELECT office_id, COUNT(*) FROM employees GROUP BY office_id;

-- How may employees are there in each office who have salary larger than 35000
-- Note: WHERE clause comes before GROUP BY
-- i.e. Filtering first, and then aggregation
SELECT office_id, COUNT(*) FROM employees WHERE salary > 35000 GROUP BY office_id;
```

#### Ways to specify a `GROUP BY` clause

- Column reference
- Grouping expressions
- Column alias (with some SQL engines)
- Two or more of the above: `GROUP BY` list

#### Choosing an aggregate function and grouping column

```MySQL
DESCRIBE inventory;
```
|name|type|
|-|-|
|shop|string|
|game|string|
|qty|int|
|aisle|tinyint|
|price|decimal(5, 2)|

---

```MySQL
-- How many different games are in stock at each shop?
SELECT shop, COUNT(*) FROM inventory GROUP BY shop;

-- How many total games are in stock at each shop?
SELECT shop, SUM(qty) FROM inventory GROUP BY shop;

-- How many total copies of each game are in stock?
SELECT game, SUM(qty) FROM inventory GROUP BY game;

-- How many total copies of each game are in stock at each shop?
-- The rows of the table already represent the grouping we are looking for.
SELECT shop, game, qty FROM inventory;
``` 

#### Grouping expressions and column alias

Besicdes using a column reference, you can use an expression in `GROUP BY` clause.

    GROPY BY boolean_expression
    GROUP BY if(boolean_expression, true_result, false_result)
    GROUP BY CASE
        WHEN ... THEN ...
        WHEN ... THEN ...
        ELSE          ...
    END
    
Note: use grouping expression in both `GROUP BY` clause and select list. This repeating can be cumbersome, so some SQL engines support *column alias*.

```SQL
-- Impala, MySQL, PostgreSQL 
-- but NOT supported in Hive
SELECT list_price < 10 AS low_price, count(*) FROM games GROUP BY low_price;
```

#### `GROUP BY` list
```SQL
SELECT min_age, max_players, COUNT(*) FROM games GROUP BY min_age, players;
SELECT min_age, list_price < 10 AS low_price, COUNT(*) FROM games GROUP BY min_age, low_price;
```

#### Grouping and aggregation, together and separately

*Aggregation without grouping* <br>
In effect, when you use an aggregate expression with no `GROPY BY` clause, there is an implict one that creates one group for the whole table.
```SQL
SELECT COUNT(*) FROM games;
```

*Grouping without aggregation*<br>
❗❗❗ When using `GROUP BY`, the `SELECT` list can only have:
- Aggregate expressions
- Expressions used in `GROUP BY`
- Literal values <br>
    
Some examples
```SQL
-- return unique values of min_ages
SELECT min_age FROM games GROUP BY min_ages;
-- better to use DISTINCT
SELECT DISTINCT min_age;
```

*Grouping with aggregation*
```SQL
SELECT min_age,
       round(AVG(list_price), 2) AS avg_list_price,
       0.21 AS tax_price,
       round(AVG(list_price) * 1.21, 2) AS avg_w_tax
  FROM games
  GROUP BY min_age;
```

### More about grouping and aggregation

#### `NULL` values in aggregation

```MySQL
-- In SQL, aggregate functions ignore NULL values
SELECT AVG(price) FROM inventory;
```

#### `NULL` values in grouping

```MySQL
-- The NULL group will be creted
SELECT aisle, COUNT(*) FROM inventory GROUP BY aisle;
```

#### Count Non-NULL values

```MySQL
/*
The COUNT function ignores NULL just like other aggregation functions
Except in the case where you use COUNT(*)
*/

-- count non-NULL values in the price column
SELECT COUNT(price) FROM inventory;

-- count non-NULL price records for each shop
SELECT shop, COUNT(price) FROM inventory GROUP BY shop;
```
---

**QUIZ**: Which statements will return the same result as this one?

```SQL
SELECT AVG(price) AS avg_price FROM fun.inventory;
```
 
```SQL
❌ A. SELECT SUM(price) / SUM(1) AS avg_price FRIM fun.inventory;
❌ B. SELECT SUM(price) / COUNT(*) AS avg_price FRIM fun.inventory;
✅ C. SELECT SUM(price) / COUNT(price) AS avg_price FRIM fun.inventory;
```
---

#### Count distinct values

```SQL
-- Still, NULL values are ignored
SELECT COUNT(DISTINCT aisle) FROM inventory;

-- Count distinct combinations
-- Works in Impala, Hive, MySQL; not PostgreSQL
SELECT COUNT(DISTINCT red, green, blue) FROM wax.crayons;

-- Mulitple distinct counts
-- Works in Hive, MySQL, PostgreSQL; not Impala
SELECT COUNT(DISTINCT red), COUNT(DISTINCT green), COUNT(DISTINCT blue) FROM wax.crayons;
```

---

**QUIZ**: Which ones will return the same as
```SQL
SELECT COUNT(tz) AS time_zones FROM fly.airports;
```

```SQL
✅ A. SELECT COUNT(*) AS time_zones FROM fly.airports WHERE tz IS NOT NULL;
❌ B. SELECT COUNT(*) AS time_zones FROM fly.airports WHERE tz is NULL;
✅ C. SELECT COUNT(ALL tz) AS time_zones FROM fly.airports;
❌ D. SELECT COUNT(*) AS time_zones FROM fly.airports;
❌ E. SELECT COUNT(DISTINCT tz) AS time_zones FROM fly.airports;
```
---


You can use `DISTINCT` with all aggregate functions
- This is rarely helpful except with `COUNT`
- With `MIN` or `MAX`, `DISTINCT` would have no effect

#### Tips for applying grouping and aggregation

1. *Pushdown* calclution to database instead of BI software. The unique values of grouping columns must be limitted (categorical).
```SQL
SELECT year, AVG(dep_delay) FROM flights GROUP BY year;
```

2. Filtering data before grouping
```SQL
SELECT dep_time, arr_time, COUNT(*) as num_flights
     FROM flights
     WHERE year=2009 AND month=1 AND day=15
     GROUP BY dep_time, arr_time;
```

3. Bining
```MySQL
-- Bining
SELECT MIN(dep_time), MAX(dep_time), COUNT(*)
     FROM flights
     GROUP BY CASE WHEN dep_time IS NULL THEN 'missing'
                   WHEN dep_time < 500 THEN 'night'
                   WHEN dep_time < 1200 THEN 'morning'
                   WHEN dep_time < 1700 THEN 'afternoon'
                   WHEN dep_time < 2200 THEN 'evening'
                   ELSE 'night'
              END;
```
```MySQL
-- using column alias
SELECT CASE WHEN dep_time IS NULL then 'missing'
            WHEN dep_time < 500 THEN 'night'
            WHEN dep_time < 1200 THEN 'morning'
            WHEN dep_time < 1700 THEN 'afternoon'
            WHEN dep_time < 2200 THEN 'evening'
            ELSE 'night'
       END AS dep_time_category,
       COUNT(*)
   FROM flights
   GROUP BY dep_time_category;
```
```MySQL
-- using positional references
SELECT CASE WHEN dep_time IS NULL then 'missing'
            WHEN dep_time < 500 THEN 'night'
            WHEN dep_time < 1200 THEN 'morning'
            WHEN dep_time < 1700 THEN 'afternoon'
            WHEN dep_time < 2200 THEN 'evening'
            ELSE 'night'
       END AS dep_time_category,
       COUNT(*)
   FROM flights
   GROUP BY 1;
```

### The `HAVING` clause

#### Filtering on aggregates

❌Invalid query
```MySQL
SELECT shop, SUM(price * qty) FROM inventory
  GROUP BY shop
  WHERE❌ SUM(price * qty) > 300;
```
The `WHERE` clause can only filter individual rows of data. Also `WHERE` clause is executed before `GROUP BY` clause.

Instead, using `HAVING` to filter aggregates ✅
```MySQL
SELECT shop, SUM(price * qty) FROM inventory
  GROUP BY shop
  HAVING SUM(price * qty) > 300;
```

Q: Which shops have at least two different games in stock that cost less than 20 dollars?
```MySQL
-- For each shop,                    -> use GROUP BY shop
-- Constraint 1: at least 2 games.   -> group constraint, use HAVING
-- Constraint 2: price < 20          -> all-row constraint, use WHERE
SELECT shop, COUNT(*) AS game_cnt
  FROM inventory
  WHERE price < 20
  GROUP BY shop
  HAVING COUNT(*) >= 2
```

Q: The `fly.planes` table contains data about plane, including the columns `manufacturer` and `seats`. Write a query will prodvide the average number of seats in all planes built by a manufacturer, but only for manufacturers who have at least one plane with more than 100 seats?

```MySQL
SELECT manufacturer, AVG(seats) FROM fly.planes GROUP BY manufacturer HAVING MAX(seats) > 100;
```

Q: The `fly.flights` table has enough information to calculate the flight speed for a flight, but it's a little long and you probably don't want to repeat it any more than you have to. The calculation for a single flight, in miles per hour, is `distance / (nullif(air_time, 0) / 60)`.

Which of the following queries for Impala is the most succinct (and correct) way to find the origin airport, destination airport, average flight speed in miles per hour, and number of flights for origin-destination pairs for which the average flight speed was over 575 miles per hour?

wrong answers omitted

```MySQL
SELECT origin, dest,
       AVG(distance / (nullif(air_time, 0) / 60)) AS avg_flight_speed,
       COUNT(*) AS num_flights
  FROM flights GROUP BY origin, dest
  HAVING avg_flight_speed > 575;
```